In [1]:
import os
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatCohere

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List
from langchain.output_parsers import OutputFixingParser
from langchain.schema import OutputParserException

In [2]:
# Define a new Pydantic model with field descriptions and tailored for Twitter.
class TravelGPT(BaseModel):
    intro: str = Field(description="Introduction about destination")
    morning: str = Field(description="Suggest things to do in the morning")
    afternoon: str = Field(description="Suggest things to do in the afternoon")
    evening: str = Field(description="Suggest things to do in the evening")
    night: str = Field(description="Suggest things to do in the night")

class DaywisePlanner(BaseModel):
    day_1: TravelGPT = Field(description="Schedule for Day 1")
    day_2: TravelGPT = Field(description="Schedule for Day 2")
    day_3: TravelGPT = Field(description="Schedule for Day 3")
    day_4: TravelGPT = Field(description="Schedule for Day 4")
    day_5: TravelGPT = Field(description="Schedule for Day 5")

In [3]:
# Instantiate the parser with the new model.
parser = PydanticOutputParser(pydantic_object=DaywisePlanner)

# Update the prompt to match the new query and desired format.
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(
            "Context : As a smart itinerary planner with extensive knowledge of places around the world, your task is to determine the user's travel destinations and any specific interests or preferences from their message. Create an itinerary that caters to the user's needs, making sure to name all activities, restaurants, and attractions specifically. When creating the itinerary, also consider factors such as time constraints and transportation options. Additionally, all attractions and restaurants listed in the itinerary must exist and be named specifically. During subsequent revisions, the itinerary can be modified, while keeping in mind the practicality of the itinerary. New place for each day. It's important to ensure that the number of activities per day is appropriate, and if the user doesn't specify otherwise, the default itinerary length is five days. The itinerary length should remain the same unless there is a change by the user's message..\n{format_instructions}\n{question}"
        )
    ],
    input_variables=["question"],
    partial_variables={
        "format_instructions": parser.get_format_instructions(),
    },
)

In [4]:
chat_model = ChatCohere()

# Generate the input using the updated prompt.
user_query = (
    "Suggest me itinerary for trip to Paris, France."
)
_input = prompt.format_prompt(question=user_query)

output = chat_model(_input.to_messages())
try:
    parsed = parser.parse(output.content)
except OutputParserException as e:
    new_parser = OutputFixingParser.from_llm(
        parser=parser,
        llm=ChatCohere()
    )
    parsed = new_parser.parse(output.content)
print(output.content)
print(parsed)

```json
{
 "day_1": {
  "intro": "Welcome to Paris, France, a city brimming with rich history, cultural attractions, and unforgettable experiences.",
  "morning": "Begin your day with a stroll along the charming streets of the Eiffel Tower neighborhood. Take in the picturesque surroundings and snap some photos of the iconic tower. Consider visiting the Musée d'Orsay, a renowned art museum housed in a converted railway station, to admire its collection of Impressionist masterpieces.",
  "afternoon": "Enjoy a leisurely lunch at a local café, sampling some of France's renowned culinary delights like croissants, baguettes, and pastries. In the afternoon, head to the Luxembourg Gardens, a tranquil green space where you can relax and unwind amidst lush greenery and beautiful fountains.",
  "evening": "For dinner, treat yourself to a gourmet dining experience at a fancy restaurant in the Le Marais district, known for its vibrant food scene and trendy eateries. Try some traditional French dish

In [8]:
parsed.day_1.morning

"Begin your day with a stroll along the charming streets of the Eiffel Tower neighborhood. Take in the picturesque surroundings and snap some photos of the iconic tower. Consider visiting the Musée d'Orsay, a renowned art museum housed in a converted railway station, to admire its collection of Impressionist masterpieces."